In [1]:
print("Akish")

Akish


In [2]:
import os
os.chdir("../")
%pwd

'c:\\Users\\akish.pothuri\\python\\LangChain\\MedicalChatBot'

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 

In [4]:
# DATA_PATH = 'data/'
# DB_FAISS_PATH = 'vectorstore/db_faiss'

In [5]:
from constants import DATA_PATH,DB_FAISS_PATH,MODEL_PATH

Create Vector DB

In [6]:
loader = DirectoryLoader(DATA_PATH,glob='*.pdf',loader_cls=PyPDFLoader)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
texts = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',model_kwargs={'device': 'cpu'})
db = FAISS.from_documents(texts, embeddings)
db.save_local(DB_FAISS_PATH)

c:\Users\akish.pothuri\python\LangChain\MedicalChatBot\medical\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model Creation

In [7]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA

Prompt

In [8]:
custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [9]:
prompt = PromptTemplate(template=custom_prompt_template,input_variables=['context', 'question'])

In [13]:
# Load the locally downloaded model here
llm = CTransformers(
    model = "model\llama-2-7b-chat.ggmlv3.q8_0.bin",
    model_type="llama",
    max_new_tokens = 512,
    temperature = 0.5
)

In [14]:
#QA Model Function
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                   model_kwargs={'device': 'cpu'})
db = FAISS.load_local(DB_FAISS_PATH, embeddings)
# qa = retrieval_qa_chain(llm, prompt, db)

In [19]:
#Retrieval QA Chain
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db.as_retriever(search_kwargs={'k': 2}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                       )

In [20]:
qa_chain("What is abdominal ultrasound?")

c:\Users\akish.pothuri\python\LangChain\MedicalChatBot\medical\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'What is abdominal ultrasound?',
 'result': 'Abdominal ultrasound is a diagnostic tool that uses sound waves to produce images of the internal organs of the abdomen, including the aorta.',
 'source_documents': [Document(page_content='Abdominal aorta ultrasound seeAbdominal\nultrasound\nAbdominal aortic aneurysm seeAortic\naneurysm\nAbdominal hernia seeHernia\nAbdominal thrust seeHeimlich maneuver\nAbdominal ultrasound\nDefinition\nUltrasound technology allows doctors to “see”', metadata={'source': 'data\\71763-gale-encyclopedia-of-medicine.-vol.-1.-2nd-ed.pdf', 'page': 14}),
  Document(page_content='In cases where ultrasound is used as a treatment tool,\npatients should educate themselves about the proposedprocedure with the help of their doctors—as is appropri-ate before any surgical procedure. Also, any abdominalultrasound procedure, diagnostic or therapeutic, may behampered by a patient’s body type or other factors, suchas the presence of excessive bowel gas (which is opaq

In [7]:
from src.prompt import Prompt
Prompt.custom_prompt_template()

"Use the following pieces of information to answer the user's question.\n                                If you don't know the answer, just say that you don't know, don't try to make up an answer.\n                                \n                                Context: {context}\n                                Question: {question}\n                                \n                                Only return the helpful answer below and nothing else.\n                                Helpful answer:\n                                "